### Load Business AR Data - Cohort 7

This will gather data, create the nanoid and load data into the Business AR database

## Setup

In [2]:
%load_ext sql
%load_ext dotenv
%config SqlMagic.named_parameters="enabled" 

In [3]:
import pandas as pd
import numpy as np
from dotenv import find_dotenv
from dotenv import load_dotenv
from nanoid import generate
load_dotenv(find_dotenv())

True

In [4]:
import string
nanoid_charset = string.ascii_letters + string.digits

In [5]:
import google.auth
credentials, project_id = google.auth.default()

In [6]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
    )
    
    return pool

## Reset Environment Variables

In [7]:
%dotenv -o

In [29]:
def get_pool(db_user, db_password, db_name, instance_connection_name):
    # function to return the database connection object
    def get_conn():
        conn = connector.connect(
            instance_connection_name,
            "pg8000",
            user=db_user,
            password=db_password,
            db=db_name
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
        echo=True,
    )
    return pool

## Create Business Connection
business_pool = get_pool(
    db_user=os.getenv("BUSINESS_USERNAME", ""),
    db_password=os.getenv("BUSINESS_PASSWORD", ""),
    db_name=os.getenv("BUSINESS_NAME", ""),
    instance_connection_name=os.getenv("BUSINESS_CONNECTION", "")
)
%sql business_pool --alias business

## Create Warehouse Connection
warehouse_pool = get_pool(
    db_user=os.getenv("WAREHOUSE_USERNAME", ""),
    db_password=os.getenv("WAREHOUSE_PASSWORD", ""),
    db_name=os.getenv("WAREHOUSE_NAME", ""),
    instance_connection_name=os.getenv("WAREHOUSE_CONNECTION", "")
)
%sql warehouse_pool --alias warehouse


2024-09-19 18:10:06,042 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-09-19 18:10:06,044 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-19 18:10:06,190 INFO sqlalchemy.engine.Engine select current_schema()
2024-09-19 18:10:06,191 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-19 18:10:06,335 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-09-19 18:10:06,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-19 18:10:07,006 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-09-19 18:10:07,007 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-19 18:10:07,154 INFO sqlalchemy.engine.Engine select current_schema()
2024-09-19 18:10:07,156 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-19 18:10:07,303 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-09-19 18:10:07,304 INFO sqlalchemy.engine.Engine [raw sql] ()


## Create Business Connection

In [30]:
# DB_USER = os.getenv("BUSINESS_USERNAME", "")
# DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
# DB_NAME = os.getenv("BUSINESS_NAME", "")
# INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

# business_pool = get_pool()
# %sql business_pool --alias business
%sql business

Switching to connection 'business'

In [12]:
%%sql
SELECT current_database(), :INSTANCE_CONNECTION_NAME as Connection, now();

Running query in 'business'

InvalidQueryParameters: Cannot execute query because the following variables are undefined: INSTANCE_CONNECTION_NAME
If you need help solving this issue, send us a message: https://ploomber.io/community


## List Database Connections

In [13]:
%sql --connections

current,url,alias
*,postgresql+pg8000://,business
,postgresql+pg8000://,warehouse


## Query Business Database

In [14]:
%sql business

In [15]:
%%sql business_data <<
select * from business;

Running query in 'business'

1573 rows affected.

In [16]:
business_data

id,legal_name,legal_type,identifier,tax_id,email,founding_date,ar_reminder_flag,last_ar_reminder_year,state,op_state,corp_class
286,S-1118 HOLDINGS LTD.,BC,BC0653374,None,cwfung@live.ca,2002-08-21 00:00:00+00:00,False,2024,ACT,None,None
443,TWIN RIVERS MOTEL LTD.,BC,BC0081392,None,crisparinvestments@gmail.com,1968-08-22 00:00:00+00:00,False,2024,ACT,None,None
652,PREMIER AUTO TRANSMISSION LTD.,BC,BC0195824,None,adil727@hotmail.com,1979-08-23 00:00:00+00:00,False,2024,ACT,None,None
2,SEA TO SKY POWERWASHING INC.,BC,BC0910741,812919801BC0001,None,None,False,None,None,None,None
133,JAZZ FOREST PRODUCTS LTD.,BC,BC0939637,None,raj@jazzforest.ca,2012-05-03 00:00:00+00:00,False,2024,ACT,None,None
134,SUNCASE NETWORKS INC.,BC,BC0942673,None,taoliu@hotmail.com,2012-06-07 00:00:00+00:00,False,2024,ACT,None,None
135,MOOZX INTERNET VENTURES INC.,BC,BC0945028,None,roland@moozx.com,2012-07-06 00:00:00+00:00,False,2024,ACT,None,None
25,THOUGHTFUL BRANDS INC.,BC,BC0885932,None,None,None,False,None,None,None,None
26,MISTYWEST ENERGY AND TRANSPORT LTD.,BC,BC0885946,None,None,None,False,None,None,None,None
27,KUBE ENTERPRISES LTD.,BC,BC0885960,None,None,None,False,None,None,None,None


## Create Warehouse Connection

In [13]:
# DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
# DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
# DB_NAME = os.getenv("WAREHOUSE_NAME", "")
# INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

# warehouse_pool = get_pool()
# %sql warehouse_pool --alias warehouse
# %sql warehouse

In [14]:
%%sql
SELECT current_database(), :INSTANCE_CONNECTION_NAME as Connection, now();

Running query in 'warehouse'

1 rows affected.

current_database,connection,now
fin_warehouse,mvnjri-prod:northamerica-northeast1:fin-warehouse-prod,2024-09-19 17:32:42.824556+00:00


## Query Warehouse Database

In [38]:
%sql warehouse

Switching to connection 'warehouse'

In [42]:
%%sql colin_data <<
SELECT co.corp_num
     , co.recognition_dts
     , EXTRACT(YEAR FROM co.last_ar_filed_dt) AS last_ar_filed_year
     , co.admin_email
     , cn.CORP_NME
     , ct.corp_class
FROM "colin"."corporation"   co
   , "colin".corp_type       ct
   , "colin".corp_state      cs
   , "colin".corp_name       cn
WHERE co.corp_typ_cd    = ct.corp_typ_cd
  AND co.corp_num       = cs.corp_num
  AND co.corp_num       = cn.corp_num
  AND cs.end_event_id   IS NULL
  and cn.end_event_id is null
  and cn.corp_name_typ_cd = 'CO'
  AND cs.state_typ_cd   = 'ACT'                                                                                -- active
  AND ct.corp_class     = 'BC'                                                                                 -- BC Corporations
  AND co.corp_typ_cd   <> 'BEN'                                                                                -- no Benefit Companies
  AND co.admin_email IS NOT NULL                                                                               -- they have an email
  AND co.send_ar_ind = 'Y'                                                                                     -- AR reminder indicator is "Y"
  AND NOT EXISTS (SELECT 'x'
                  FROM "colin".filing f, "colin".event e, "colin".filing_user u
                  WHERE f.event_id = e.event_id
                    AND f.event_id = u.event_id                                                                -- no previous BCOL filings
                    AND e.corp_num = co.corp_num
                    AND u.role_typ_cd = 'bcol')
  AND NOT EXISTS (SELECT 'x'
                  FROM "colin".corporation
                  WHERE admin_email = co.admin_email
                    AND corp_num <> co.corp_num)                                                               -- no other business using the same email
  AND (date_part('doy', co.recognition_dts) BETWEEN date_part('doy', current_date)                            -- AR reminder within the next 14 days based on the day of year
       AND date_part('doy', current_date))                                      -- AR reminder on the anniversary date
  AND (
       -- Exclude companies founded in the current year, include those from previous year
       (EXTRACT(YEAR FROM co.recognition_dts) = EXTRACT(YEAR FROM current_date) - 1 AND co.last_ar_filed_dt IS NULL)
       -- Or include if last_ar_filed_dt is not NULL and was filed in the previous year
       OR (co.last_ar_filed_dt IS NOT NULL AND EXTRACT(YEAR FROM co.last_ar_filed_dt) = EXTRACT(YEAR FROM current_date) - 1)
      );

Running query in 'warehouse'

2024-09-19 18:28:08,192 INFO sqlalchemy.engine.Engine SELECT co.corp_num
     , co.recognition_dts
     , EXTRACT(YEAR FROM co.last_ar_filed_dt) AS last_ar_filed_year
     , co.admin_email
     , cn.CORP_NME
     , ct.corp_class
FROM "colin"."corporation"   co
   , "colin".corp_type       ct
   , "colin".corp_state      cs
   , "colin".corp_name       cn
WHERE co.corp_typ_cd    = ct.corp_typ_cd
  AND co.corp_num       = cs.corp_num
  AND co.corp_num       = cn.corp_num
  AND cs.end_event_id   IS NULL
  and cn.end_event_id is null
  and cn.corp_name_typ_cd = 'CO'
  AND cs.state_typ_cd   = 'ACT'
  AND ct.corp_class     = 'BC'
  AND co.corp_typ_cd   <> 'BEN'
  AND co.admin_email IS NOT NULL
  AND co.send_ar_ind = 'Y'
  AND NOT EXISTS (SELECT 'x'
                  FROM "colin".filing f, "colin".event e, "colin".filing_user u
                  WHERE f.event_id = e.event_id
                    AND f.event_id = u.event_id
                    AND e.corp_num = co.corp_num
                    AN

124 rows affected.

In [43]:
colin_df = colin_data.DataFrame()
colin_df

,corp_num,recognition_dts,last_ar_filed_year,admin_email,corp_nme,corp_class
0,0614385,2000-09-19 00:00:00,2023,geg@telus.net,GG OILFIELD SERVICES LTD.,BC
1,1379419,2022-09-20 10:17:02,2023,nswong@hotmail.com,ALIYA HOLDING LIMITED,BC
2,1440313,2023-09-20 12:44:08,None,farokh.hassani@gmail.com,NEXPERT AUTO CORP.,BC
3,0634198,2001-09-20 00:00:00,2023,iat@askiat.com,IAT INDUSTRIAL AUTOMATION TECHNOLOGY INC.,BC
4,0981007,2013-09-20 15:12:57,2023,LifeGeoPro@gmail.com,UNFARM LIFE-GEO INC.,BC
...,...,...,...,...,...,...
119,0920650,2011-09-20 08:27:01,2023,claire@synapseconsulting.ca,SYNAPSE CONSULTING INC.,BC
120,0980954,2013-09-20 12:16:05,2023,vkk@shaw.ca,VANCOUVER SEIYU KARATE INC.,BC
121,1324991,2021-09-20 13:18:43,2023,ich830paul@gmail.com,ECO H&C CLEANING LTD.,BC
122,1379539,2022-09-20 17:45:44,2023,aashishkohli@hotmail.com,AASHISH KOHLI LAW CORPORATION,BC


In [44]:
# Print original columns to verify names
print("Original columns:", colin_df.columns.tolist())

# Rename columns (adjusted based on actual column names)
colin_df.rename(columns={
    'corp_num': 'identifier',
    'corp_nme': 'legal_name',  # Adjusted to match actual column name
    'recognition_dts': 'founding_date',
    'admin_email': 'email',
    'last_ar_filed_year': 'last_ar_reminder_year',
    'corp_class': 'legal_type'  # Use 'corp_class' as 'legal_type'
}, inplace=True)

# Print columns after renaming to confirm
print("Columns after renaming:", colin_df.columns.tolist())

# Replace 'None' strings with actual None values
colin_df.replace('None', None, inplace=True)

# Convert data types
colin_df['last_ar_reminder_year'] = pd.to_numeric(colin_df['last_ar_reminder_year'], errors='coerce').astype('Int64')
colin_df['founding_date'] = pd.to_datetime(colin_df['founding_date'], errors='coerce')

# Select the required columns
colin_df = colin_df[['identifier', 'legal_name', 'legal_type', 'founding_date', 'last_ar_reminder_year', 'email']]

# Function to escape and format values for SQL
def escape_and_format(value):
    if isinstance(value, str):
        # Escape single quotes in strings
        return "'" + value.replace("'", "''") + "'"
    elif pd.isnull(value):
        # Handle None values
        return 'NULL'
    elif isinstance(value, pd.Timestamp):
        # Format dates as 'YYYY-MM-DD'
        return "'" + value.strftime('%Y-%m-%d') + "'"
    else:
        # For other data types, convert to string or handle None
        return str(int(value)) if value is not None else 'NULL'

# Generate the VALUES part of the INSERT statement
values = ",\n".join([
    "(" + ", ".join([
        escape_and_format(row['identifier']),
        escape_and_format(row['legal_name']),
        escape_and_format(row['legal_type']),
        escape_and_format(row['founding_date']),
        escape_and_format(row['last_ar_reminder_year']),
        escape_and_format(row['email'])
    ]) + ")"
    for _, row in colin_df.iterrows()
])

# Complete the INSERT statement
insert_statement = f"""INSERT INTO "public"."business" (identifier, legal_name, legal_type, founding_date, last_ar_reminder_year, email)
VALUES 
{values};"""

# Print the INSERT statement
print(insert_statement)


Original columns: ['corp_num', 'recognition_dts', 'last_ar_filed_year', 'admin_email', 'corp_nme', 'corp_class']
Columns after renaming: ['identifier', 'founding_date', 'last_ar_reminder_year', 'email', 'legal_name', 'legal_type']
INSERT INTO "public"."business" (identifier, legal_name, legal_type, founding_date, last_ar_reminder_year, email)
VALUES 
('0614385', 'GG OILFIELD SERVICES LTD.', 'BC', '2000-09-19', 2023, 'geg@telus.net'),
('1379419', 'ALIYA HOLDING LIMITED', 'BC', '2022-09-20', 2023, 'nswong@hotmail.com'),
('1440313', 'NEXPERT AUTO CORP.', 'BC', '2023-09-20', NULL, 'farokh.hassani@gmail.com'),
('0634198', 'IAT INDUSTRIAL AUTOMATION TECHNOLOGY INC.', 'BC', '2001-09-20', 2023, 'iat@askiat.com'),
('0981007', 'UNFARM LIFE-GEO INC.', 'BC', '2013-09-20', 2023, 'LifeGeoPro@gmail.com'),
('1224052', 'SAVOCO SOLUTIONS INC.', 'BC', '2019-09-20', 2023, 'savocosolutions@gmail.com'),
('1180225', 'VANITY GRACE FURNISHINGS LTD.', 'BC', '2018-09-20', 2023, 'info@vanitygrace.com'),
('0102935

In [45]:
# Verify connection by querying the current database
%sql business


Switching to connection 'business'

In [27]:
%%sql
SELECT * from business;

Running query in 'business'

1573 rows affected.

id,legal_name,legal_type,identifier,tax_id,email,founding_date,ar_reminder_flag,last_ar_reminder_year,state,op_state,corp_class
286,S-1118 HOLDINGS LTD.,BC,BC0653374,None,cwfung@live.ca,2002-08-21 00:00:00+00:00,False,2024,ACT,None,None
443,TWIN RIVERS MOTEL LTD.,BC,BC0081392,None,crisparinvestments@gmail.com,1968-08-22 00:00:00+00:00,False,2024,ACT,None,None
652,PREMIER AUTO TRANSMISSION LTD.,BC,BC0195824,None,adil727@hotmail.com,1979-08-23 00:00:00+00:00,False,2024,ACT,None,None
2,SEA TO SKY POWERWASHING INC.,BC,BC0910741,812919801BC0001,None,None,False,None,None,None,None
133,JAZZ FOREST PRODUCTS LTD.,BC,BC0939637,None,raj@jazzforest.ca,2012-05-03 00:00:00+00:00,False,2024,ACT,None,None
134,SUNCASE NETWORKS INC.,BC,BC0942673,None,taoliu@hotmail.com,2012-06-07 00:00:00+00:00,False,2024,ACT,None,None
135,MOOZX INTERNET VENTURES INC.,BC,BC0945028,None,roland@moozx.com,2012-07-06 00:00:00+00:00,False,2024,ACT,None,None
25,THOUGHTFUL BRANDS INC.,BC,BC0885932,None,None,None,False,None,None,None,None
26,MISTYWEST ENERGY AND TRANSPORT LTD.,BC,BC0885946,None,None,None,False,None,None,None,None
27,KUBE ENTERPRISES LTD.,BC,BC0885960,None,None,None,False,None,None,None,None


In [46]:
try:
    colin_df.to_sql(name='business', con=business_pool, if_exists='append', index=False)
except Exception as e:
    print("An error occurred during to_sql operation:", e)


2024-09-19 18:28:30,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-19 18:28:30,227 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %s::VARCHAR AND pg_catalog.pg_class.relkind = ANY (ARRAY[%s::VARCHAR, %s::VARCHAR, %s::VARCHAR, %s::VARCHAR, %s::VARCHAR]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %s::VARCHAR
2024-09-19 18:28:30,228 INFO sqlalchemy.engine.Engine [cached since 1075s ago] ('business', 'r', 'p', 'f', 'v', 'm', 'pg_catalog')
2024-09-19 18:28:30,519 INFO sqlalchemy.engine.Engine INSERT INTO business (identifier, legal_name, legal_type, founding_date, last_ar_reminder_year, email) VALUES (%s::VARCHAR, %s::VARCHAR, %s::VARCHAR, %s::TIMESTAMP WITHOUT TIME ZONE, %s::BIGINT, %s::VARCHAR)
2024-09-19 18:28:30,519 INFO sqlalchemy.engine.Engine [

In [82]:
# Close the connection and reconnect
business_pool.dispose()  # Disposes the current pool and closes all connections
business_pool = get_pool()  # Recreate the connection pool


In [1]:
colin_df

NameError: name 'colin_df' is not defined

## Load from CSV

In [ ]:
colin_df = pd.read_csv('cohort_7.csv') 
colin_df

In [ ]:
# colin_df["identifier"] = colin_df["Jurisdiction"] + colin_df['corp_num'].apply('{:0>7}'.format)
# colin_df['nano_id'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
# colin_df['tax_id'] = None
# colin_df['id'] = np.arange(1, colin_df.shape[0] + 1) + 7
#to_bar_df = pd.DataFrame()
#to_bar_df[['id','legal_name','legal_type','identifier','tax_id','nano_id']] = colin_df[['id','Name','Jurisdiction','identifier','tax_id','nano_id']]
#to_bar_df


In [ ]:
# colin_df['tax_id'] = None
# colin_df['ar_reminder_flag'] = None
# colin_df['state'] = None
# colin_df['op_state'] = None
# colin_df['corp_class'] = None
to_bar_df = colin_df

In [ ]:
to_bar_df.to_sql(name='b2', con=business_pool, if_exists='append', index=False)

1

In [ ]:
%%sql
INSERT INTO business (id, legal_name, legal_type, identifier, email, founding_date, last_ar_reminder_year, state)
SELECT 
    (SELECT COALESCE(MAX(id), 0) FROM business) + ROW_NUMBER() OVER (), 
    legal_name, 
    legal_type, 
    identifier, 
    email, 
    founding_date::date, 
    last_ar_reminder_year, 
    'ACT'
FROM b2;


In [ ]:
to_inv_email_df = pd.DataFrame()
to_inv_email_df[['id','recipients']] = colin_df[['id','email']]
to_inv_email_df['business_id'] = to_inv_email_df['id']
to_inv_email_df['token'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
to_inv_email_df['message'] = colin_df.apply(lambda row: 'AR Invitation', axis = 1)
to_inv_email_df['status'] = colin_df.apply(lambda row: 'SENT', axis = 1)
to_inv_email_df

In [ ]:
%%sql business_data <<
select * from business where id = 9

## If using csv files then no need to run following two cells

In [ ]:
business_df = business_data.DataFrame()

In [ ]:
to_inv_email_df = pd.DataFrame()
to_inv_email_df[['id','recipients']] = business_df[['id','email']]
to_inv_email_df['business_id'] = to_inv_email_df['id']
to_inv_email_df['token'] = business_df.apply(lambda row: generate(nanoid_charset), axis = 1)
to_inv_email_df['message'] = business_df.apply(lambda row: 'AR Invitation', axis = 1)
to_inv_email_df['status'] = business_df.apply(lambda row: 'SENT', axis = 1)
to_inv_email_df

Writes into inv_ch6 table

In [ ]:
to_inv_email_df.to_sql(name='inv_ch6', con=business_pool, if_exists='append', index=False)

1

Insert into invitations table from inv_ch6 table

In [ ]:
%%sql
insert into invitations (id,recipients,message,sent_date,token,status,business_id)
(select nextval('invitations_id_seq'::regclass), recipients, message, now(), token, 'SENT', business_id from inv_ch6)

## Output to CSV

In [ ]:
%%sql business_data <<
select b.identifier, b.legal_name, b.legal_type, i.token, i.recipients from business b, invitations i where b.id=i.business_id and b.id >= 104

In [ ]:
business_data

In [ ]:
bdf = business_data.DataFrame()
# bdf = bdf[['legal_name','legal_type','identifier','nano_id']]
base_url='https://annualreport.business.bcregistry.gov.bc.ca/en-CA'
bdf['url'] = f'{base_url}?nanoid=' + bdf['token']
bdf

In [ ]:
import time
from datetime import datetime
from datetime import timezone

time_stamp = time.time()
now = datetime.utcfromtimestamp(time_stamp).replace(tzinfo=timezone.utc)
# local_now = now.astimezone(Pacific)
local_now = now.astimezone()
local_now.strftime("%Y.%m.%d.%H")

In [ ]:
with open('business-ar.'+local_now.strftime("%Y.%m.%d.%H")+'.csv', 'a') as f:      
    f.write('\n\n Business Annual Report\n')
    bdf.to_csv(f, sep=',', encoding='utf-8', index=False)    

## Set Invitations

In [ ]:
%%sql
INSERT INTO invitations (id, recipients, message, sent_date, token, status, additional_message, business_id)
SELECT 
    nextval('invitations_id_seq'::regclass), 
    'test@example.com', 
    'Annual Report Due', 
    now(), 
    b2.nano_id,
    'SENT', 
    NULL, 
    b.id
FROM 
    business b
JOIN 
    b2 ON b.id = b2.id;
